## Read .ghg metadata

In [73]:
import os
import time
import numpy as np
import pandas as pd
import datetime as dt
# import py

T1 = time.time()


def find_all_GHG(site):
    raw_path = f'C:/highfreq/{site}/raw/'
    all_files = []
    all_paths = []

    for (root, dir, files) in os.walk(raw_path):
        if root != raw_path:
            for file in files:
                name, tag = file.split('.')
                # .ghg files are located at the end of each directory tree
                # Avoids reading any that might be misplaced elsewhere
                if tag == 'ghg' and len(dir)==0:
                    all_files.append(file)
                    all_paths.append(root.replace(raw_path,''))

    df = pd.DataFrame(data={
            'filename':all_files,
            'path':all_paths
            })
    
    # df['datetime'] = pd.to_datetime(df['filename'].str.split('_').str[0])
    # df['clean_tv'] = dt.datetime(1,1,1)# - df['datetime']
    return (df)

df = find_all_GHG('BB2')

# print(time.time()-T1)

# df = df.set_index('datetime')
# # df.loc[]
# df['Flag'] = 'O'
# df.loc[((df.index.minute!=30)&(df.index.minute!=0)),'Flag'] = 'S'
# flagged = df.loc[df['Flag']=='S']
# flagged.index.date
# flag_dates=pd.to_datetime(flagged.groupby(flagged.index.date).first().index)#.date
# flag_dates


# df = df.loc[df['Flag']!='S'].resample('30T').first()
# df.loc[df['Flag']!='O']
# df.loc[df['filename'] is None]
df

ModuleNotFoundError: No module named 'py'

In [36]:
for file in files:
    try:
        
        name, tag = file.split('.')
    except:
        print(dir)
        print(file)

['2019', '2020', '2021', '2022', '2023']
2022-05-19T110000_AIU-1696.data.part
['2019', '2020', '2021', '2022', '2023']
2022-05-19T110000_AIU-1696.metadata.part
['2019', '2020', '2021', '2022', '2023']
2022-05-19T110000_AIU-1696.status.part
['2019', '2020', '2021', '2022', '2023']
ziHqk4m9
['2019', '2020', '2021', '2022', '2023']
ziKvbXad


In [12]:
import os
import time
import pandas as pd
from Parse_GHG import read_GHG
from ipywidgets import FloatProgress
from IPython.display import display

# Site = 'BB2'

# Values we expect to change
dynamicMetaData_Headers = ['altitude','canopy_height']

# Values we want to fix
staticMetaData_FloatHeaders = ['altitude', 'latitude', 'longitude']
staticMetaData_TextHeaders = ['site_name']

T1 = time.time()
print('Processing ghg files ')
f = FloatProgress(min=0, max=1) 
display(f) 
# Walk through the directory to find all "raw" folders

N_recs = 4#8*31
def find_new_GHG(Site,make_new=False):

    i = 0
    Dpath = 'C:\\highfreq\\'
    raw_dir = Dpath+Site+'\\raw\\'
    meta_dir = Dpath+Site+'\\metadata\\'
    meta_file = meta_dir+'GHGMetaData.csv'
    channel_file = meta_dir+'EP_Channels.csv'

    if os.path.isfile(meta_file) and make_new is False:
        Records = pd.read_csv(meta_file,parse_dates=['TimeStamp'])
        Channels = pd.read_csv(channel_file)#,parse_dates=['TimeStamp'])
        FileNames = Records['filename'].tolist()
    else:
        Records = pd.DataFrame()
        Channels = pd.DataFrame()
        FileNames = []
        if not os.path.exists(meta_dir):
            os.mkdir(meta_dir)
    

    for (root, dirs, files) in sorted(os.walk(raw_dir)):
        # if root.__contains__('raw') :
        for file in files:
            name, tag = file.split('.')
            # .ghg files are located at the end of each directory tree
            # Avoids reading any that might be misplaced elsewhere
            if tag == 'ghg' and len(dirs)==0 and file not in FileNames:  
                
                # if i <= 48*5:
                if i <= 2:
                    GHG_out = read_GHG(root,name)
                    Records = pd.concat(
                        [Records,GHG_out.Summary],
                        axis=0,
                        ignore_index=True
                    )
                    Channels = pd.concat(
                        [Channels,GHG_out.Channels],
                        axis=0,
                        ignore_index=True
                    )
                    i += 1
            f.value = i/len(files)
    return(Records,Channels,meta_dir,i)

Records,Channels,meta_dir,i = find_new_GHG('BB',make_new=False)

Records.to_csv(meta_dir+'GHGMetaData.csv',index=False)
Channels.to_csv(meta_dir+'EP_Channels.csv',index=False)

elapsed = time.time()-T1
print('Processing time ', elapsed) 
print('To inspect ', i, ' .ghg files')
print('Seconds/file ', elapsed/(i+1)) 

staticMetaData = Records[staticMetaData_FloatHeaders].astype(float).mean()
staticMetaData = pd.concat([Records[staticMetaData_TextHeaders].astype(str).mode().T[0],staticMetaData])
print(staticMetaData)

dynamicMetaData = Records.set_index('TimeStamp')[dynamicMetaData_Headers]
dynamicMetaData['date']=dynamicMetaData.index.date
dynamicMetaData['time']=dynamicMetaData.index.time
dynamicMetaData.to_csv(meta_dir+'dynamicMetaData.csv',index=False)
dynamicMetaData

# Processing time  65.87000775337219
# To inspect  241  .ghg files

Records['altitude'].astype(float).describe()


Processing ghg files 


FloatProgress(value=0.0, max=1.0)

Processing time  1.290328025817871
To inspect  3  .ghg files
Seconds/file  0.3225820064544678
site_name    BurnsBog2018
altitude             -3.8
latitude        49.129353
longitude     -122.984885
dtype: object


count    6.000000
mean    -3.800000
std      3.935988
min     -7.400000
25%     -7.225000
50%     -4.800000
75%     -0.125000
max      0.800000
Name: altitude, dtype: float64

In [20]:
# Channels = Channels.set_index('filename')
Channels.loc[(Channels.diff()).sum(axis=1)!=0].index


# Channels[(Channels.diff()!=0).sum(axis=0)]
# Channels.diff()f

Index([], dtype='object', name='filename')

In [72]:
import os
# import struct
import numpy as np
import pandas as pd
# print(os.cwd)

os.getcwd()

Dir = 'W:\\2022\\BB\\Flux\\'

Trace = 'co2_mean'
Time = 'clean_tv'

data_file = Dir + Trace
time_file = Dir + Time

# with open(filename, mode='rb') as file:
    # if Trace == 'clean_tv':
Time = np.fromfile(open(time_file, mode='rb'), 'float64')
    # else:
Var = np.fromfile(open(data_file, mode='rb'), 'float32')
# # print(Var.shape,Time.shape)

# df = pd.DataFrame(data = {
#     'Timestamp':Time,
#     'Var':Var
# })

print(Var.shape)
print(2*24*365)

# print(Time.shape)
# Time = Time.reshape(-1,2)
# Time
# pd.to_datetime(Time,unit='s')

    
#     Data = np.array(struct.unpack("i" * ((len(fileContent) -24) // 4), fileContent[20:-4]))

print(Time)
# 738522.02083333
# Matlab Epoch Base ('0-1-0')
base_time = 719529#.000001
pd.to_datetime(Time-base_time,unit='D').round('s')

(17520,)
17520
[738522.02083333 738522.04166667 738522.0625     ... 738886.95833333
 738886.97916667 738887.        ]


DatetimeIndex(['2022-01-01 00:30:00', '2022-01-01 01:00:00',
               '2022-01-01 01:30:00', '2022-01-01 02:00:00',
               '2022-01-01 02:30:00', '2022-01-01 03:00:00',
               '2022-01-01 03:30:00', '2022-01-01 04:00:00',
               '2022-01-01 04:30:00', '2022-01-01 05:00:00',
               ...
               '2022-12-31 19:30:00', '2022-12-31 20:00:00',
               '2022-12-31 20:30:00', '2022-12-31 21:00:00',
               '2022-12-31 21:30:00', '2022-12-31 22:00:00',
               '2022-12-31 22:30:00', '2022-12-31 23:00:00',
               '2022-12-31 23:30:00', '2023-01-01 00:00:00'],
              dtype='datetime64[ns]', length=17520, freq=None)

2460026.1590252155